In [14]:
import tensorflow as tf
import numpy as np

In [ ]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

In [26]:
batch_size = 32
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=12354
)

raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=12354
)

raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/test",
    batch_size=batch_size
)

print(f"Number of batches in raw_train_ds: {raw_train_ds.cardinality()}")
print(f"Number of batches in raw_val_ds: {raw_val_ds.cardinality()}")
print(f"Number of batches in raw_test_ds: {raw_test_ds.cardinality()}")

Found 75000 files belonging to 3 classes.
Using 60000 files for training.
Found 75000 files belonging to 3 classes.
Using 15000 files for validation.
Found 25000 files belonging to 2 classes.
Number of batches in raw_train_ds: 1875
Number of batches in raw_val_ds: 469
Number of batches in raw_test_ds: 782


In [5]:
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(5):
        print(text_batch.numpy()[i])
        print(label_batch.numpy()[i])

b"First, a warning. 'How to Marry a Millionaire' comes prefaced by an apparently random five minute orchestral performance of 'Street Scene', a Gershwin-lite piece treated with the full pomp and ceremony of, well, Gershwin. Sitting through it takes some patience. If you have the DVD, rest assured, you can skip forward. You won't miss anything.<br /><br />The film itself is one of the perpetual disappointments of 50's Hollywood, a movie so bolstered by major star-power, opulent mise-en-scene and perfect high-concept that failure seems inconceivable. The title alone is perfect. Generation after generation, however, are forced to ask themselves - how is this so limp? The script is an albatross about the production's neck, a dead, smelling thing that chokes everything and everyone before they can really spark to life. There are no comic situations, just isolated moments that play for laughs. Whenever an actual comedy scene threatens to develop, the movie quickly moves on to other, less int

In [20]:
from tensorflow.python.keras import layers
import string 
import re # regular expression

In [8]:
string.punctuation

# html tags and punctuation removal from text 
def custom_standardization(input_data):  
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, f"[{re.escape(string.punctuation)}]", ""
    )

In [9]:
# Model constants.
max_features = 20000 # Number of words to use.
embedding_dim = 128 # Dimension of the embedding vector.
sequence_length = 500 # cut off the text after 500 words

In [35]:
# # Vectorize the text samples into a 2D integer tensor of shape [batch_size, sequence_length]
# vectorize_layer = tf.keras.layers.TextVectorization(
#     standardize=custom_standardization,
#     max_tokens=max_features,
#     output_mode="int",
#     output_sequence_length=sequence_length
# )

In [36]:
def vectorize_text(text, label): # vectorize the text and label into a 2D integer tensor of shape [batch_size, sequence_length]
    text = tf.expand_dims(text, -1) # Add a dimension to the end of the tensor.
    return vectorize_layer(text), label # Return the vectorized text and the label.

# Vectorize the data.
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

# Do async prefetching / buffering of the data for best performance on GPU.
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)

In [37]:
from tensorflow.keras import layers

# A integer input for vocab indices.
inputs = tf.keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

model = tf.keras.Model(inputs, predictions)

# Compile the model with the Adam optimizer and binary crossentropy loss.
# Adam is a gradient-based optimizer that is good for stochastic gradient descent.
# Binary crossentropy is a loss function that can be used to train a binary classifier.
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [ ]:
epochs = 3

# Fit the model using the train and test datasets.
model.fit(train_ds, validation_data=val_ds, epochs=epochs)

In [ ]:
model.evaluate(test_ds)

In [34]:
# A string input
inputs = tf.keras.Input(shape=(1,), dtype="string")
# Turn strings into vocab indices
indices = vectorize_layer(inputs)
# Turn vocab indices into predictions
outputs = model(indices)

NameError: name 'model' is not defined

In [ ]:
# Our end to end model
end_to_end_model = tf.keras.Model(inputs, outputs)
end_to_end_model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
)

# Test it with `raw_test_ds`, which yields raw strings
end_to_end_model.evaluate(raw_test_ds)